LGBM Parameter : <https://www.kaggle.com/hamzaghanmi/lgbm-hyperparameter-tuning-using-optuna>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import datatable as dt
import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

from lightgbm import LGBMRegressor

In [ ]:
train = dt.fread('../input/tabular-playground-series-feb-2021/train.csv').to_pandas()
test = dt.fread('../input/tabular-playground-series-feb-2021/test.csv').to_pandas()

In [ ]:
cat_col = [c for c in train.columns if 'cat' in c]
cont_col = [c for c in train.columns if 'cont' in c]

In [ ]:
for c in cat_col:
    le = LabelEncoder()
    train[c] = le.fit_transform(train[c])
    test[c] = le.transform(test[c])

In [ ]:
kfold = KFold(5, True, random_state = 87)

In [ ]:
X = train[cat_col + cont_col]
y = train['target']
X_test = test[cat_col + cont_col]

In [ ]:
lgbm_params = {
    'bagging_freq': 1,
    'reg_alpha': 2.4766410381355457,
    'reg_lambda': 2.644144282261626,
    'colsample_bytree': 0.3,
    'subsample': 0.6,
    'learning_rate': 0.008,
    'max_depth': 20,
    'num_leaves': 139,
    'min_child_samples': 176,
    'random_state': 48,
    'n_estimators': 20000,
    'metric': 'rmse',
    'cat_smooth': 9}

In [ ]:
results = np.zeros(X_test.shape[0])
models = []
loss = []
num = 1

for tr, te in kfold.split(X, y):
    print(f'{num} Fold Start')
    X_train, X_val = X.iloc[tr], X.iloc[te]
    y_train, y_val = y.iloc[tr], y.iloc[te]
    model = LGBMRegressor(**lgbm_params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), eval_metric = 'rmse', early_stopping_rounds = 500, verbose=500)
    results += model.predict(X_test)
    loss.append(model.best_score_['valid_0']['rmse'])
    models.append(model)
    num += 1
    
results = results / len(models)
loss = np.mean(loss)

In [ ]:
# 0.8422742689096191 lgbm
print(loss)

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')
submission['target'] = results
submission

In [ ]:
now = datetime.datetime.now().strftime('%Y-%m-%d:%H:%M')
submission.to_csv(f'./{now}_submission.csv', index= False)